In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sympy as sp
from scipy.integrate import cumulative_simpson as cumsimp


In [ ]:
class CovarianceFunction:
    @staticmethod
    def square_exponential(x, sigma=1, l=1):
        """The square exponential covariance function for 1D input."""
        n = len(x)
        K = np.array([[sigma**2 * np.exp(-(x[i] - x[j])**2 / (2 * l**2)) for j in range(n)] for i in range(n)])
        return K

class EOSData:
    def __init__(self, crust_file, mean_file):
        self.crust_data = np.loadtxt(crust_file)
        self.mean_data = np.loadtxt(mean_file)
        
        # Unpacking data
        self.n_crust, self.e_crust, self.p_crust, self.cs2_crust = self.crust_data.T
        self.n_mean, self.e_mean, self.p_mean, self.cs2_mean = self.mean_data.T
        self.mu_ini = (self.e_crust[-1] + self.p_crust[-1]) / self.n_crust[-1]

class GPEstimation:
    def __init__(self, eos_data, rng=np.random.default_rng()):
        self.eos_data = eos_data
        self.rng = rng
        
    def generate_phi(self, log_p, sigma=1, l=1):
        covP = CovarianceFunction.square_exponential(log_p, sigma, l)
        phi = self.rng.multivariate_normal(np.zeros(len(log_p)), cov=covP)
        return phi

    def calculate_epsilon(self, phi, pressure):
        d_epsilon_dp = np.exp(phi) + 1
        epsilon = cumsimp(y=d_epsilon_dp, x=pressure, initial=self.eos_data.e_crust[-1])
        return epsilon

class EOSProperties:
    @staticmethod
    def sound_speed_squared(phi=None, epsilon=None, pressure=None, c=1):
        """Compute the sound speed squared."""
        if epsilon is None and phi is not None:
            return 1 / (1 + np.exp(phi))
        elif epsilon is not None and pressure is not None:
            return np.gradient(pressure, epsilon) * c
        else:
            raise ValueError('You need to provide either epsilon or phi')
    
    @staticmethod
    def chemical_potential(epsilon, pressure, c_s2, mu_init):
        """Compute the chemical potential."""
        integrand = c_s2 / (epsilon + pressure)
        integral = cumsimp(y=integrand, x=epsilon, initial=np.log(mu_init))
        ln_mu = integral
        return ln_mu
    
    @staticmethod
    def number_density(epsilon, pressure, ln_mu):
        """Compute the number density."""
        mu = np.exp(ln_mu)
        n = (epsilon + pressure) / mu
        return n

# Example Usage
# Initialize EOS data
eos_data = EOSData('EOS/crust/crust_eos.dat', 'EOS/mean/36022_microscopic.dat')

# Logarithmic pressure array
p = np.logspace(np.log10(eos_data.p_crust[-1]), np.log10(2000), 100)
log_p = np.log(p)

# Generate GP samples
gp = GPEstimation(eos_data)
phi = gp.generate_phi(log_p)

# Plot GP
plt.plot(log_p, phi)
plt.xlabel(r'$\log(p)$')
plt.ylabel(r'$\phi$')
plt.show()

# Compute epsilon
epsilon = gp.calculate_epsilon(phi, p)

# Plot epsilon
plt.plot(p, epsilon)
plt.xlabel(r'$p$')
plt.ylabel(r'$\epsilon$')
plt.show()

# Compute sound speed squared
c_s2 = EOSProperties.sound_speed_squared(phi=phi)

# Compute chemical potential
ln_mu = EOSProperties.chemical_potential(epsilon, p, c_s2, eos_data.mu_ini)

# Compute number density
n = EOSProperties.number_density(epsilon, p, ln_mu)
